<a href="https://colab.research.google.com/github/tuvaom/Exercise1/blob/master/hw1_naive_bayes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

You can do this homework online through [Google Colab](https://colab.research.google.com/). By linking Colab to Github, you can import this file to colab, and save your changes back to Github directly.


# Homework 1: Naive Bayes Classifier

In this homework you will implement the Naive Bayes Classsifier on a data set of votes in the U.S. House of Representatives, with the goal of predicting the party affiliation of each congressman. The input data $X$ is given by a $N$-by-$D$ matrix, where $N$ is the number of examples and $D=16$ is the number of input features. Each feature is binary (yes/no). The targets are given by a length-$N$ sequence of classes, $Y$, that are also binary. More information on the data set can be found at  https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records.



First, we need to download the data. The following code uses the urllib library to request data from a website. The pandas library is a powerful library for data analysis --- we use the read_csv method to automatically parse the comma seperated variable (csv) file.

In [0]:
import pandas 
import urllib.request  
import numpy   # Numerical python.
from sklearn.metrics import log_loss

# Download the data.
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data"
response = urllib.request.urlopen(url)

# Interpret text data into pandas data frame. Interpret 'abstain' votes as 'no'.
dataset  = pandas.read_csv(response, header=None, true_values=['y'], false_values=['n','?'])

# Set the column names.
names = ['label'] + [f'vote_{i}' for i in range(16)]
dataset.columns = names

# Tells pandas that this is a categorical feature.
dataset['label'] = pandas.Categorical(dataset['label'])

print("Dataset shape: ", dataset.shape)
dataset.head() # Prints first 5 examples from the data set.



Dataset shape:  (435, 17)


,label,vote_0,vote_1,vote_2,vote_3,vote_4,vote_5,vote_6,vote_7,vote_8,vote_9,vote_10,vote_11,vote_12,vote_13,vote_14,vote_15
0,republican,False,True,False,True,True,True,False,False,False,True,False,True,True,True,False,True
1,republican,False,True,False,True,True,True,False,False,False,False,False,True,True,True,False,False
2,democrat,False,True,True,False,True,True,False,False,False,False,True,False,True,True,False,False
3,democrat,False,True,True,False,False,True,False,False,False,False,True,False,True,False,False,True
4,democrat,True,True,True,False,True,True,False,False,False,False,True,False,True,True,True,True


Numpy is a powerful library for mathematical operations on vectors and matrices. Here we convert the pandas data into a 2-dimensional numpy array (a matrix). 

In [0]:
X = numpy.array(dataset.iloc[:,1:]) # Convert input features into Numpy array.
Y = dataset['label'].cat.codes # Converts string labels to binary values.

# Split data into train and test set. Use the first 335 examples for training.
num_train = 335
Xtrain = X[0:num_train, :]
Xtest  = X[num_train:,:]
print(Xtrain.shape, Xtest.shape)

Ytrain = Y[0:num_train]
Ytest = Y[num_train:]
print(Xtrain[0][1],Xtrain[0][2])
print(Ytrain)



(335, 16) (100, 16)
True False
0      1
1      1
2      0
3      0
4      0
      ..
330    1
331    0
332    0
333    0
334    0
Length: 335, dtype: int8


You are asked to implement the following functions.

In [0]:
def generative_model(Xtrain, Ytrain):
    ''' 
    Implements a generative algorithm on binary data.
    Inputs
        Xtrain: NxD matrix of features.
        Ytrain: N vector of class labels
    Returns
        p_label: Length 2 vector of class probabilities.
        p_votes: 2xD Matrix where entry i,j is p(x_j|v=i).
    ''' 
    N = Xtrain.shape[0]
    D = Xtrain.shape[1]

    assert(len(Xtrain)==len(Ytrain))
    # vector of class probabilities
    p_label_count = [0,0]
    for i in range(N):
      if Ytrain[i] == 0:
        p_label_count[0] += 1
      else:
        p_label_count[1] += 1
    total = p_label_count[0] + p_label_count[1]
    p_label = [p_label_count[0]/total, p_label_count[1]/total]

    # probability of class given vote
    p_votes = [[0]*16, [0]*16]

    for i in range(D):
      no = [0,0]
      yes = [0,0]
      for j in range(N):
        if Xtrain[j][i] == False:
          no[Ytrain[j]] += 1
        else:
          yes[Ytrain[j]] += 1
      total_no = no[0] + no[1]
      total_yes = yes[0] + yes[1]
      p_votes[0][i] = no[1]/total_no    #probability for republican if answer no
      p_votes[1][i] = yes[1]/total_yes   #probability for republican if answer yes
      

    return p_label, p_votes

  

def discriminative_model(p_label, p_votes, Xtest):
    '''
    Implements Naive Bayes Classification.
    Inputs
      p_label, p_votes: From generative_model.
      Xtest: NxD matrix of binary features.
    
    Outputs 
      pred_prob: Probability of label=1 under model.
    '''
    # Probability for Republican
    N = Xtest.shape[0]
    D = Xtest.shape[1]
    probD = p_label[0]
    probR = p_label[1]
    pred_prob = [0]*N
    for i in range(N):
      prob_votes_given_R =1
      prob_votes_given_D =1
      for j in range(D):
        if Xtest[i][j] == False:
          vote = 0
        else:
          vote = 1
        prob_votes_given_R= prob_votes_given_R*p_votes[vote][j]
        prob_votes_given_D = prob_votes_given_D*(1-p_votes[vote][j])

      #prob_votes_given_D= 1-prob_votes_given_R
      pred_prob[i]=(prob_votes_given_R*probR)/(prob_votes_given_D*probD+prob_votes_given_R*probR)
    return pred_prob




Ytest.reset_index(drop=True, inplace=True)    #index from 0

def accuracy(y_true, y_predicted):
    ''' Calculates the fraction of correct predictions.
    '''
    assert(len(y_true) == len(y_predicted))
    # WRITE ME
    N = len(y_true)
    total=0

    for i in range(N):
      total += abs(y_true[i] - y_predicted[i])
    dev = total/N
    acc = 1-dev

    return acc

  


In [0]:
# Make sure to print these for submission.
p_label, p_votes = generative_model(Xtrain, Ytrain)
print('Label priors:', p_label)
print('Conditional vote probabilities:', p_votes)

pred_prob = discriminative_model(p_label, p_votes, Xtest)
print('Predictions:', pred_prob)



acc = accuracy(Ytest, pred_prob)  
ll = log_loss(Ytest, pred_prob)
print("Accuracy: ", acc)
print("logloss: ", ll)

fifty_prob = [0.5]*len(Ytest)

fifty_acc = accuracy(Ytest, fifty_prob)
fifty_ll = log_loss(Ytest, fifty_prob)

print("50/50 accuracy: ", fifty_acc)
print("50/50 logloss:", fifty_ll)

Label priors: [0.6208955223880597, 0.37910447761194027]
Conditional vote probabilities: [[0.5520833333333334, 0.3541666666666667, 0.8248175182481752, 0.01951219512195122, 0.03867403314917127, 0.1223021582733813, 0.6783216783216783, 0.7655172413793103, 0.6845238095238095, 0.3372093023255814, 0.5, 0.1111111111111111, 0.13259668508287292, 0.04794520547945205, 0.6010362694300518, 0.44274809160305345], [0.14685314685314685, 0.4125874125874126, 0.0707070707070707, 0.9461538461538461, 0.7792207792207793, 0.5612244897959183, 0.15625, 0.08421052631578947, 0.0718562874251497, 0.4233128834355828, 0.10679611650485436, 0.8125, 0.6688311688311688, 0.6349206349206349, 0.07746478873239436, 0.3382352941176471]]
Predictions: [0.9997284333397235, 1.8493345705928362e-09, 1.3551592226813938e-14, 6.752076210856877e-13, 0.9972015124400774, 0.994452164500896, 1.5365488466777087e-13, 2.4727571248461085e-08, 0.014547680928305986, 1.6739451667981512e-14, 0.8800730485856323, 0.9999495144432301, 0.9997674875435352

## To turn in:
1) Implement the Naive Bayes Classifier using the starter code above. Make sure to print out the parameters of the generative model and the predictions on the test points.

2) Compute the log probability of the test set --- this is a single scalar value.

0.965

3) Compare the NB classifier to a model in which we predict a 50-50 chance for each vote, in terms of accuracy and the log probability. Which model is better and why? Describe two situations in which the Naive Bayes Classifier will fail. 


better results with the NB classifier. 
The Naive Bayes would fail with a sorted list, or all from one party.


